In [ ]:
import torch
from torch import nn
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [ ]:
# use correct device

device = 'cuda' if torch.cuda.is_available else 'cpu'

In [ ]:
# load training data

df_train = pd.read_csv('train.csv')
df_train.head(5)

In [ ]:
# load further data

df_test = pd.read_csv('test.csv')
df_test.head()

In [ ]:
# handle date

"""
df_train['datetime'] = pd.to_datetime(df_train['first_day_of_month'])
df_train = df_train.set_index('datetime')
df_train.drop(['first_day_of_month'], axis=1, inplace=True)
df_train.head(10)
"""

In [ ]:
# add time index as column

df_train["time_index"] = df_train.groupby("cfips").cumcount() + 1
df_train.head()

In [ ]:
# extract data used

data = df_train[['time_index','cfips','microbusiness_density']]

In [ ]:
# extract label

data['y'] = data['microbusiness_density'].shift(-1)

In [ ]:
# remove missing values

data = data.dropna()

In [ ]:
# sort data

data=data.sort_values(by=['cfips','time_index'], ascending=True)
data.tail()

In [ ]:
# train-test-split

train=data[data.time_index<=30]
test=data[data.time_index>30]
train.head()

In [ ]:
train_X = train.drop(['y', 'cfips'], axis=1)
train_y = train['y']
test_X = test.drop(['y', 'cfips'], axis=1)
test_y = test['y']

test_X.head()

In [ ]:
# Train-Test-Split

training_size = int(len(train_X) * 0.8)
test_size = len(train_X) - training_size

train_data = train_X[0:training_size].to_numpy()
test_data = train_X[training_size:].to_numpy()

In [ ]:
class MLTSDataset(Dataset):
    def __init__(self, data, seq_len = 1):
        self.data = data
        self.data = torch.from_numpy(data).float()
        self.seq_len = seq_len
        
    def __len__(self):
        return len(self.data)-self.seq_len-1
    
    def __getitem__(self, index):
        return self.data[index:index+self.seq_len], self.data[index+self.seq_len]

In [ ]:
train_dataset = MLTSDataset(train_data)
test_dataset = MLTSDataset(test_data)

In [ ]:
# Maybe try other implementation again, seemed to work better...

class BiGRU(nn.Module):
    def __init__(self, input_dim, hidden_size, num_layers):
        super(BiGRU, self).__init__()
        self.num_layers = num_layers
        self.input_dim = input_dim
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size=input_dim, hidden_size = hidden_size, num_layers=num_layers, bidirectional=True)
        self.fc = nn.linear(hidden_size*2, 1)
        
    def forward(self, x, hn):
        out, hn = self.BiGRU()
        final_out = self.fc(out[-1])
        return final_out

In [ ]:
# Change those values again

input_dim = len(train_data.shape)
hidden_size = 25
num_layers = 12
batch_size = 64

model = BiGRU(input_dim, hidden_size, num_layers).to(device)

In [ ]:
optimizer = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
def train(dataloader):
    hn = torch.zeros(num_layers, batch_size, slef.hidden_size).to(device)
    model.train()
    
    for batch, item in enumerate(dataloader):
        x = x.to(device)
        y = y.to(device)
        
        # prediction
        out, hn = model(x.reshape[100, batch_size, 1], hn)
        loss = optimizer(out.reshape(batch_size), y)
        hn = hn.detach()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch == len(dataloder)-1:
            loss = loss.item()
            print('Train Loss:' + loss)

In [ ]:
"""

"""class BiGRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(BiGRU, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.output_size = output_size
        
        # create bilinear GRU with n layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x, seq_len):
        out, h_n = self.gru(x)
        h_n = torch.cat((h_n[-2,:,:], h_n[-1,:,:]), dim=1)
        h_n = h_n.repeat(seq_len, 1, 1)
        x_pred = torch.zeros_like(x)
        x_pred[:,:seq_len,:] = x
        for t in range(seq_len):
            out, h_n = self.gru(x_pred[:,t:t+1,:], h_n)
            x_pred[:,t+1,:] = self.fc(out[:, -1, :])
        return x_pred[:,seq_len:,:]